## `03_get_variant_cluster`: Get variants called from the reads mapped back to the assembled contigs

The assembled contigs should be consistent with the reads that used to generate them. We mapped the reads back to the contigs and check if there are clusters of variants to find potenting issues of assembly or consensus errors.

In [ ]:
%%bash
dx cd /20200316_asm_for_revision/
dx download -f H1_pa_phase_ctg.fa H2_pa_phase_ctg.fa
dx download -f data/H1.fa data/H2.fa data/unphased.fa
dx download -f data/MHC_37.fa

In [ ]:
%%capture
%%bash
. /opt/conda/bin/activate
conda install -c bioconda -y freebayes=1.3.1
conda install -c bioconda -y minimap2 samtools pbmm2 pbsv

In [ ]:
%%bash
. /opt/conda/bin/activate
cat H1.fa unphased.fa > H1_and_unphased.fa

pbmm2 align H1_pa_phase_ctg.fa H1_and_unphased.fa --sort --preset CCS --sample sample1 --rg '@RG\tID:movie1' > H1_to_H1asm.bam 
samtools index H1_to_H1asm.bam
cat H2.fa unphased.fa > H2_and_unphased.fa

pbmm2 align H2_pa_phase_ctg.fa H2_and_unphased.fa --sort --preset CCS --sample sample1 --rg '@RG\tID:movie1' > H2_to_H2asm.bam
samtools index H2_to_H2asm.bam

In [ ]:
%%bash
samtools faidx -f H1_pa_phase_ctg.fa
samtools faidx -f H2_pa_phase_ctg.fa

In [ ]:
%%bash 
# this takes about one hours to run
freebayes-parallel <(fasta_generate_regions.py H1_pa_phase_ctg.fa.fai 500000) 8 \
   -G 10 -F 0.3  -f H1_pa_phase_ctg.fa H1_to_H1asm.bam > var_H1_2.vcf &

freebayes-parallel <(fasta_generate_regions.py H2_pa_phase_ctg.fa.fai 500000) 8 \
   -G 10 -F 0.3  -f H2_pa_phase_ctg.fa H2_to_H2asm.bam > var_H2_2.vcf &


In [ ]:
%%bash
bgzip -f var_H1_2.vcf
bgzip -f var_H2_2.vcf


In [ ]:
%%bash
tabix -p vcf var_H1_2.vcf.gz
tabix -p vcf var_H2_2.vcf.gz

In [ ]:
%%bash
pbsv discover H1_to_H1asm.bam H1_to_H1asm.svsig.gz
pbsv discover H2_to_H2asm.bam H2_to_H2asm.svsig.gz
pbsv call H1_pa_phase_ctg.fa H1_to_H1asm.svsig.gz H1_to_H1asm.sv.vcf
pbsv call H2_pa_phase_ctg.fa H2_to_H2asm.svsig.gz H2_to_H2asm.sv.vcf
bgzip -f H1_to_H1asm.sv.vcf 
bgzip -f H2_to_H2asm.sv.vcf

In [ ]:
%%bash
dx cd /20200316_asm_for_revision
dx upload var_H1_2.vcf.gz
dx upload var_H2_2.vcf.gz
dx upload H2_to_H2asm.sv.vcf.gz
dx upload H2_to_H2asm.sv.vcf.gz 

In [ ]:
import gzip

f = gzip.open("var_H1_2.vcf.gz")
v_pos = []
for row in f:
    if row[0] == ord(b'#'):
        continue
    row = row.strip().split()
    if row[0] != b'000000F':
        continue
    if len(row[3]) != len(row[4]): # only count for SNP
        continue
    position = int(row[1])
    v_pos.append(position)

f = gzip.open("H1_to_H1asm.sv.vcf.gz") # from pb_sv call 
for row in f:
    if row[0] == ord(b'#'):
        continue
    row = row.strip().split()
    if row[0] != b'000000F':
        continue
    position = int(row[1])
    k, v = row[-2:]
    k = k.split(b':')
    v = v.split(b':')
    d = dict(zip(k,v))
    if b'AD' not in d:
        continue
    AD = [int(_) for _ in d[b'AD'].split(b',')]
    if min(AD)/sum(AD) < 0.25:
        continue
    pos = int(row[1])
    v_pos.append(position)

v_pos.sort()
SNP_cluster=[]
cluster_d = 10000
for position in v_pos:
    if len(SNP_cluster) == 0 or position - SNP_cluster[-1][-1] > cluster_d:
        SNP_cluster.append([position])
    else:
        SNP_cluster[-1].append(position)
f = open("H1_exclude.bed","w")
for c in SNP_cluster:
    #print("H1", len(c), '000000F:{}-{}'.format(min(c)-50, max(c)+50), max(c)-min(c)+100)
    print('000000F\t{}\t{}'.format(min(c)-50, max(c)+50), file=f)
f.close()
for c in SNP_cluster:
    print((min(c)-50, max(c)+50), max(c)-min(c)+100)

In [ ]:
%%bash
apt-get install zlib1g-dev
git clone https://github.com/lh3/seqtk.git;
cd seqtk; make

In [ ]:
%%bash
./seqtk/seqtk subseq H1_pa_phase_ctg.fa H1_exclude.bed > out.fa
grep ">" out.fa
minimap2 -xasm5 MHC_37.fa  out.fa > out.paf
cat out.paf | awk '$12 >0 {print 6" "$8+28477798" "$9+28477798" H1:"$1":"$12}' | sort -k2 -g > H1_exclude_GRCH37.bed

In [ ]:
!cat H1_exclude_GRCH37.bed

In [ ]:
f = gzip.open("var_H2_2.vcf.gz")
v_pos = []
for row in f:
    if row[0] == ord(b'#'):
        continue
    row = row.strip().split()
    if row[0] != b'000000F':
        continue
    if len(row[3]) != len(row[4]):
        continue
    position = int(row[1])
    v_pos.append(position)

f = gzip.open("H2_to_H2asm.sv.vcf.gz")
for row in f:
    if row[0] == ord(b'#'):
        continue
    row = row.strip().split()
    if row[0] != b'000000F':
        continue
    position = int(row[1])
    k, v = row[-2:]
    k = k.split(b':')
    v = v.split(b':')
    d = dict(zip(k,v))
    if b'AD' not in d:
        continue
    AD = [int(_) for _ in d[b'AD'].split(b',')]
    if min(AD)/sum(AD) < 0.25:
        continue
    pos = int(row[1])
    v_pos.append(position)

v_pos.sort()
SNP_cluster=[]
cluster_d = 10000
for position in v_pos:
    if len(SNP_cluster) == 0 or position - SNP_cluster[-1][-1] > cluster_d:
        SNP_cluster.append([position])
    else:
        SNP_cluster[-1].append(position)
f = open("H2_exclude.bed","w")
for c in SNP_cluster:
    #print("H1", len(c), '000000F:{}-{}'.format(min(c)-50, max(c)+50), max(c)-min(c)+100)
    print('000000F\t{}\t{}'.format(min(c)-50, max(c)+50), file=f)
f.close()
for c in SNP_cluster:
    print((min(c)-50, max(c)+50), max(c)-min(c)+100)

In [ ]:
%%bash
./seqtk/seqtk subseq H2_pa_phase_ctg.fa H2_exclude.bed > out.fa
grep ">" out.fa
minimap2 -xasm5 MHC_37.fa  out.fa > out.paf
cat out.paf | awk '$12 >0 {print 6" "$8+28477798" "$9+28477798" H2:"$1":"$12}' | sort -k2 -g  > H2_exclude_GRCH37.bed

In [ ]:
!cat H2_exclude_GRCH37.bed

In [ ]:
!cat H1_exclude_GRCH37.bed H2_exclude_GRCH37.bed | sort -k 2 -g  | tee exclude_GRCH37.bed

In [ ]:
%%bash
dx cd /20200316_asm_for_revision
dx upload exclude_GRCH37.bed
dx upload H2_exclude_GRCH37.bed H1_exclude_GRCH37.bed